In [1]:
# Instalaciones
!pip install pytorch-pretrained-bert
!pip install seqeval
!pip install langdetect


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 9.0 MB/s 
     |████████████████████████████████| 132 kB 57.1 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 9.1 MB 57.9 MB/s 
     |████████████████████████████████| 140 kB 68.5 MB/s 
     |████████████████████████████████| 127 kB 57.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=36d28225e6592de3b0c9150e9c4a7da7d415dac4e01e0d69a4e6d5544f9aa513
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b35

In [2]:
# Importaciones
# GENERAL Y PREPROCESADO
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import spacy
!python -m spacy download es_core_news_sm
import es_core_news_sm
from langdetect import detect
import os.path
import datetime
# BERT
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from seqeval.metrics import f1_score


from IPython.display import clear_output 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 8.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
# Conexion a drive y descompresión de los corpus
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/COLAB\ -\ TFM/corpora-master.zip

Archive:  /content/drive/MyDrive/COLAB - TFM/corpora-master.zip
370a87a85d1d24d418a86abbe1f6918383362467
   creating: corpora-master/
  inflating: corpora-master/.gitignore  
   creating: corpora-master/.vscode/
  inflating: corpora-master/.vscode/settings.json  
   creating: corpora-master/2021/
  inflating: corpora-master/2021/README.md  
   creating: corpora-master/2021/eval/
   creating: corpora-master/2021/eval/develop/
  inflating: corpora-master/2021/eval/develop/README.md  
   creating: corpora-master/2021/eval/develop/scenario1-main/
 extracting: corpora-master/2021/eval/develop/scenario1-main/input.ann  
  inflating: corpora-master/2021/eval/develop/scenario1-main/input.txt  
  inflating: corpora-master/2021/eval/develop/scenario1-main/output.ann  
  inflating: corpora-master/2021/eval/develop/scenario1-main/output.txt  
   creating: corpora-master/2021/eval/develop/scenario2-taskA/
 extracting: corpora-master/2021/eval/develop/scenario2-taskA/input.ann  
  inflating: corpora

## Funciones

In [44]:
def clasifica_data(file):
  file_csv = file
  ingles = []
  espanol = []
  with open(file_csv, "r") as f:
    texto = f.read().split("\n")
  for frase in texto:
    espaciado = frase.split(" ")
    idioma = detect(frase)
    if idioma == "en":
      ingles.append(frase)
    elif idioma == "es":
      espanol.append(frase)
  return ingles, espanol

  
def data_preprocessing_es(texto, ann = None):
  nlp = es_core_news_sm.load()
  nlp_sentences = [nlp(x) for x in texto]
  list_registro = []
  for i, frase in enumerate(nlp_sentences):
    for j, token in enumerate(frase):
      list_registro.append([i, j, token.lower_, token.pos_])
  df_train = pd.DataFrame(list_registro)
  df_train.columns = ["Sentence #", "Word In Sentence #", "Word", "POS"]
  df_train = df_train[~df_train["Word"].isin([" ",""])][df_train["POS"] != "PUNCT"]
  if ann != None:
    with open(ann, "r") as f:
      texto_ann = [x.split("\t") for x in f.read().split("\n")]
    df_ann = pd.DataFrame(texto_ann)
    df_ann.columns = ["num_termino", "desc", "Word"]
    df_ann_terminos = df_ann[df_ann["num_termino"].str.contains("T")].copy()
    df_ann_terminos["TipoEntidad"] = df_ann_terminos["desc"].str.split(" ").apply(lambda x: x[0])
    df_ann_terminos["Word_list"] = df_ann_terminos["Word"].apply(lambda x: list(nlp(x)))
    df_ann_terminos = df_ann_terminos.explode("Word_list")
    df_ann_terminos["rn"] = df_ann_terminos.groupby("num_termino")["Word"].cumcount()+1
    df_ann_terminos.loc[:,"Tag"]  = "I-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"Tag"] = "B-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos = df_ann_terminos[~df_ann_terminos["Word_list"].isin([" ",""])]
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # Algoritmo de anotacion
    # Nos quedamos con un unico registro por numero de termino, palabra y tag
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # generamos un flag unico
    df_ann_cruce["unico"] = df_ann_cruce["num_termino"] + df_ann_cruce["Word"].\
                              apply(lambda x: str(x))
    # inicializamos el pasado a NO
    df_ann_cruce["pasado"] = "NO"
    # Inicializamos la ventana de busqueda
    ventana_busqueda = 20
    registro = []
    df_train_preproceso = df_train.copy()
    # Incializamos el ultimo termino encontrado a -1 para evitar descartar ningun termino de partida
    max_termino = -1
    # Iteramos por cada palabra del corpus y buscaremos en el dataframe de anotacion
    for num_linea, linea in enumerate(df_train_preproceso.iterrows()):
      palabra = linea[1]["Word"]
      # Definimos el dataframe de busqueda
      df_lookup = df_ann_cruce.head(ventana_busqueda)
      # Nos quedamos con el registro encontrado
      ann_match = df_lookup[df_lookup["Word"].\
                              apply(lambda x: str(x).lower()).\
                              str.replace(".","") == palabra.replace(".","")].head(1)
      # if linea[1]["Sentence #"] == 724:
      #   print(palabra)
      #   print(df_lookup)
      #   print(ann_match)
      #   print(num_linea)
      #   print(df_ann_cruce)
      if len(ann_match) > 0:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                      linea[1]["POS"], ann_match["Tag"].iloc[0], ann_match["num_termino"].iloc[0]])
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["unico"] != ann_match["unico"].iloc[0],:]
        max_termino = ann_match["num_termino"].iloc[0]
        # print(len(df_ann_cruce), print(len(registro)))
      else:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                          linea[1]["POS"], "O", "NA"])
      if num_linea % 100 == 0 and num_linea > 0:
        df_ann_cruce.loc[df_ann_cruce["num_termino"].apply(lambda x: int(x[1:])) < int(max_termino[1:]),"pasado"] = "SI"
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["pasado"] != "SI",:]
    df = pd.DataFrame(registro)
    df.columns = ["Sentence #", "Word", "POS", "Tag", "num_concepto"]
    data = df[["Sentence #", "Word", "POS", "Tag"]]
    MAX_LEN = int(data["Sentence #"].value_counts().quantile(0.95))
    df_tmp = (data["Sentence #"].value_counts() <= MAX_LEN)
    data = data[data["Sentence #"].isin(df_tmp[df_tmp].index.tolist())]
    return data, MAX_LEN
  else:
    return df_train[["Word", "POS"]]


def data_preprocessing_en(texto, ann = None):
  nlp = spacy.load("en_core_web_sm")
  nlp_sentences = [nlp(x) for x in texto]
  list_registro = []
  for i, frase in enumerate(nlp_sentences):
    for j, token in enumerate(frase):
      list_registro.append([i, j, token.lower_, token.pos_])
  df_train = pd.DataFrame(list_registro)
  df_train.columns = ["Sentence #", "Word In Sentence #", "Word", "POS"]
  df_train = df_train[~df_train["Word"].isin([" ",""])][df_train["POS"] != "PUNCT"]
  if ann != None:
    with open(ann, "r") as f:
      texto_ann = [x.split("\t") for x in f.read().split("\n")]
    df_ann = pd.DataFrame(texto_ann)
    df_ann.columns = ["num_termino", "desc", "Word"]
    df_ann_terminos = df_ann[df_ann["num_termino"].str.contains("T")].copy()
    df_ann_terminos["TipoEntidad"] = df_ann_terminos["desc"].str.split(" ").apply(lambda x: x[0])
    df_ann_terminos["Word_list"] = df_ann_terminos["Word"].apply(lambda x: list(nlp(x)))
    df_ann_terminos = df_ann_terminos.explode("Word_list")
    df_ann_terminos["rn"] = df_ann_terminos.groupby("num_termino")["Word"].cumcount()+1
    df_ann_terminos.loc[:,"Tag"]  = "I-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"Tag"] = "B-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos = df_ann_terminos[~df_ann_terminos["Word_list"].isin([" ",""])]
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # Algoritmo de anotacion
    # Nos quedamos con un unico registro por numero de termino, palabra y tag
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # generamos un flag unico
    df_ann_cruce["unico"] = df_ann_cruce["num_termino"] + df_ann_cruce["Word"].\
                              apply(lambda x: str(x))
    # inicializamos el pasado a NO
    df_ann_cruce["pasado"] = "NO"
    # Inicializamos la ventana de busqueda
    ventana_busqueda = 20
    registro = []
    df_train_preproceso = df_train.copy()
    # Incializamos el ultimo termino encontrado a -1 para evitar descartar ningun termino de partida
    max_termino = -1
    # Iteramos por cada palabra del corpus y buscaremos en el dataframe de anotacion
    for num_linea, linea in enumerate(df_train_preproceso.iterrows()):
      palabra = linea[1]["Word"]
      # Definimos el dataframe de busqueda
      df_lookup = df_ann_cruce.head(ventana_busqueda)
      # Nos quedamos con el registro encontrado
      ann_match = df_lookup[df_lookup["Word"].\
                              apply(lambda x: str(x).lower()).\
                              str.replace(".","") == palabra.replace(".","")].head(1)
      # if linea[1]["Sentence #"] == 724:
      #   print(palabra)
      #   print(df_lookup)
      #   print(ann_match)
      #   print(num_linea)
      #   print(df_ann_cruce)
      if len(ann_match) > 0:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                      linea[1]["POS"], ann_match["Tag"].iloc[0], ann_match["num_termino"].iloc[0]])
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["unico"] != ann_match["unico"].iloc[0],:]
        max_termino = ann_match["num_termino"].iloc[0]
        # print(len(df_ann_cruce), print(len(registro)))
      else:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                          linea[1]["POS"], "O", "NA"])
      if num_linea % 100 == 0 and num_linea > 0:
        df_ann_cruce.loc[df_ann_cruce["num_termino"].apply(lambda x: int(x[1:])) < int(max_termino[1:]),"pasado"] = "SI"
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["pasado"] != "SI",:]
    df = pd.DataFrame(registro)
    df.columns = ["Sentence #", "Word", "POS", "Tag", "num_concepto"]
    data = df[["Sentence #", "Word", "POS", "Tag"]]
    MAX_LEN = int(data["Sentence #"].value_counts().quantile(0.95))
    df_tmp = (data["Sentence #"].value_counts() <= MAX_LEN)
    data = data[data["Sentence #"].isin(df_tmp[df_tmp].index.tolist())]
    return data, MAX_LEN
  else:
    return df_train[["Word", "POS"]]


class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

def bert_formating(data, MAX_LEN, test_size=0.2, bs = 16):
  #concat sentence
  getter = SentenceGetter(data)
  word_list = [ [s[0] for s in sent] for sent in getter.sentences] 
  sentences = word_list
  labels = [[s[2] for s in sent] for sent in getter.sentences]
  tags_vals = list(set(data["Tag"].values))
  tag2idx = {t: i for i, t in enumerate(tags_vals)}
  idx2tag = {i: t for i, t in enumerate(tags_vals) }
  words = list(set(data["Word"].values))
  n_words = len(words); 
  word2idx = {w: i + 2 for i, w in enumerate(words)}
  word2idx["UNK"] = 1
  word2idx["PAD"] = 0
  idx2word = {i: w for w, i in word2idx.items()}
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  if device == torch.device("cuda"):
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0) 
  tokenized_texts = word_list
  tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
  input_ids = pad_sequences(tokens_ids,
                          maxlen=int(MAX_LEN), dtype="int64", truncating="post", padding="post")
  for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("need more max_len - defect after filtering")
        MAX_LEN = len(i)
  t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
  tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")
  attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
  #split train test
  tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                              random_state=2022, test_size=test_size)
  tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                              random_state=2022, test_size=test_size)
  tr_inputs = torch.tensor(tr_inputs)
  val_inputs = torch.tensor(val_inputs)
  tr_tags = torch.tensor(tr_tags)
  val_tags = torch.tensor(val_tags)
  tr_masks = torch.tensor(tr_masks)
  val_masks = torch.tensor(val_masks)
  train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

  valid_data = TensorDataset(val_inputs, val_masks, val_tags)
  valid_sampler = SequentialSampler(valid_data)
  valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)
  return train_data, train_sampler, train_dataloader, valid_data, valid_sampler,\
            valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs

def bert_formating_one_dataset(data, MAX_LEN, bs = 16, tag2idx = None, tags_vals=None,word2idx=None):
  #concat sentence
  getter = SentenceGetter(data)
  word_list = [ [s[0] for s in sent] for sent in getter.sentences] 
  sentences = word_list
  labels = [[s[2] for s in sent] for sent in getter.sentences]
  if tags_vals is None:
    tags_vals = list(set(data["Tag"].values))
  if tag2idx is None:
    tag2idx = {t: i for i, t in enumerate(tags_vals)}
  words = list(set(data["Word"].values))
  n_words = len(words)
  if word2idx is None:
    word2idx = {w: i + 2 for i, w in enumerate(words)}
    word2idx["UNK"] = 1
    word2idx["PAD"] = 0
  idx2word = {i: w for w, i in word2idx.items()}
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  if device == torch.device("cuda"):
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0) 
  tokenized_texts = word_list
  # tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]

  tokens_ids = []
  for s in tokenized_texts:
    tmp = []
    for w in s:
      try:
        tmp.append(word2idx[w])
      except:
        tmp.append(word2idx["UNK"])
    tokens_ids.append(tmp)
  print(tokens_ids)
  input_ids = pad_sequences(tokens_ids,
                          maxlen=int(MAX_LEN), dtype="int64", truncating="post", padding="post")
  for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("need more max_len - defect after filtering")
        MAX_LEN = len(i)
  t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
  tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")
  attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
  #split train test
  tr_inputs = torch.tensor(input_ids, dtype=torch.int64)
  tr_tags = torch.tensor(tags, dtype=torch.int64)
  tr_masks = torch.tensor(attention_masks, dtype = torch.uint8)

  train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)
  return train_data, train_sampler, train_dataloader, tag2idx, device, tags_vals, idx2word, tr_inputs, word2idx
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= False, save_path = ""):
  model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))
  if device == torch.device("cuda"):
    model.cuda()
  if reentreno:
    print("Loading existing model...")
    model.load_state_dict(torch.load(save_path))
  if FULL_FINETUNING:
      param_optimizer = list(model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.0}
      ]
  else:
      param_optimizer = list(model.classifier.named_parameters()) 
      optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
  optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)
  return model, optimizer

def training(model, optimizer, train_dataloader, valid_dataloader, tags_vals, idx2word, epochs = 200,
             max_grad_norm = 1.0, save_path = "./bert1", early_stopping=15, use_scheduler=True):
  lmbda = lambda x: 0.98 if use_scheduler else 1
  scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
  train_f1 = []
  train_losses = []
  val_losses = []
  contador_max = -1
  for _ in trange(epochs, desc="Epoch"):
      # TRAIN loop
      model.train()
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      for step, batch in enumerate(train_dataloader):
          # add batch to gpu
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          # forward pass
          loss = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask, labels=b_labels)
          # backward pass
          loss.backward()
          # track train loss
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
          # gradient clipping
          # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
          # update parameters
          optimizer.step()
          model.zero_grad()
      scheduler.step()
      # print train loss per epoch
      print("Train loss: {}".format(tr_loss/nb_tr_steps))
      # VALIDATION on validation set
      model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0
      predictions , true_labels, true_inputs = [], [],[]
      for batch in valid_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch          
          with torch.no_grad():
              tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                    attention_mask=b_input_mask, labels=b_labels)
              logits = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          inputs = b_input_ids.to('cpu').numpy()
          
          true_inputs.append(inputs)
          predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
          true_labels.append(label_ids)
          
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          
          eval_loss += tmp_eval_loss.mean().item()
          eval_accuracy += tmp_eval_accuracy
          
          nb_eval_examples += b_input_ids.size(0)
          nb_eval_steps += 1
      eval_loss = eval_loss/nb_eval_steps
      print("Validation loss: {}".format(eval_loss))
      print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
      pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
      valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
      valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]
      f1 = f1_score([pred_tags], [valid_tags])
      train_f1.append(f1)
      train_losses.append(tr_loss/nb_tr_steps)
      val_losses.append(eval_loss)

      max_f1 = max(train_f1)
      if f1 == max_f1:
        contador_max = 1
        torch.save(model.state_dict(), save_path)
      if contador_max > 0:
        contador_max += 1
      print("F1-Score: " + str(train_f1[-1]))
      if round(max_f1, 2) > 0.1 and contador_max > early_stopping :
        print("Early stopping...")
        return 0
def training_scheduler(model, optimizer, loss_fn, train_dataloader, valid_dataloader, tags_vals, idx2word, epochs=200,
             max_grad_norm=1.0, save_path="./bert1", early_stopping=15, print_freq=10, use_scheduler=True):
    lmbda = lambda x: 0.98 if use_scheduler else 1
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

    train_f1 = []
    train_losses = []
    val_losses = []
    contador_max = -1

    for ep in range(epochs):
        # TRAIN loop
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(train_dataloader):
            # add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            # forward pass
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            loss = model.compute_loss(loss_fn, logits, b_labels, b_input_mask)
            # backward pass
            loss.backward()
            # track train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
            # gradient clipping
            # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            # update parameters
            optimizer.step()
            model.zero_grad()
        scheduler.step()
        # print train loss per epoch
        if ep % print_freq == 0:
            print(f"{ep}/{epochs}: Train loss: {tr_loss / nb_tr_steps}")

        # VALIDATION on validation set
        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels, true_inputs = [], [], []
        for batch in valid_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            tmp_eval_loss = model.compute_loss(loss_fn, logits, b_labels, b_input_mask)

            logits = logits.cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            inputs = b_input_ids.to('cpu').numpy()

            true_inputs.append(inputs)
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)

            tmp_eval_accuracy = flat_accuracy(logits, label_ids)

            eval_loss += tmp_eval_loss.item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        eval_loss = eval_loss / nb_eval_steps
        if ep % print_freq == 0:
            print("\tValidation loss: {}".format(eval_loss))
            print("\tValidation Accuracy: {}".format(eval_accuracy / nb_eval_steps))
        pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
        valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in true_inputs for l_i in l]
        f1 = f1_score([pred_tags], [valid_tags])
        train_f1.append(f1)
        train_losses.append(tr_loss / nb_tr_steps)
        val_losses.append(eval_loss)

        max_f1 = max(train_f1)
        if f1 == max_f1:
            contador_max = 1
            torch.save(model.state_dict(), save_path)
        if contador_max > 0:
            contador_max += 1
        if ep % print_freq == 0:
            print(f"\tF1-Score: {train_f1[-1]}\n")

        if round(max_f1, 2) > 0.1 and contador_max > early_stopping:
            print("We should apply early stopping now")

def evaluate(model, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = "./bert1",
             guarda_resultado="/content/drive/MyDrive/COLAB - TFM/resultado_entrenamiento1.csv"):
  #evaluate model
  model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))
  model.load_state_dict(torch.load(save_path))
  if device == torch.device("cuda"):
    model.cuda()  
  model.eval()
  predictions = []
  true_labels = []
  true_inputs = []

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  print(len(valid_dataloader))
  for batch in tqdm(valid_dataloader):
      #print(len(batch))
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      with torch.no_grad():
          tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels)
          logits = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
          
      logits = logits.detach().cpu().numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      label_ids = b_labels.to('cpu').numpy()
      inputs = b_input_ids.to('cpu').numpy()
      true_inputs.append(inputs)
      
      
      true_labels.append(label_ids)
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)

      eval_loss += tmp_eval_loss.mean().item()
      eval_accuracy += tmp_eval_accuracy

      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1

  pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
  valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
  valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]

  count = 0 
  all_data = 0
  registro_resultado = []
  for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
      for k,l,kk in zip(i,j,ll):
        count += 1
        registro_resultado.append([k,l,idx2word[kk.item()]])
        all_data += 1
  df_alldata = pd.DataFrame(registro_resultado)
  df_alldata.columns = ["pred_tags", "valid_tags", "val_inputs"]
  df_alldata[df_alldata["val_inputs"] != "PAD"].\
        to_csv(guarda_resultado,sep=";", encoding = "utf-8")
  return pred_tags, valid_tags, valid_inputs




## Entrenamiento

In [45]:
raiz = "/content/drive/MyDrive/COLAB - TFM/"
sufijo = "_preprocessing.csv"
archivos = ["wikinews.25.txt","wikinews.75.txt","wikinews.300.es.txt",
            "medline.25.txt","medline.75.txt","medline.1200.es.txt",
            "cord.50.txt","cord.150.txt"]
for i, archivo in enumerate(archivos):
  if i == 0:
    df = pd.read_csv(raiz+archivo+sufijo, index_col=0)
    df["fichero"] = archivo.split(".")[1]
    df["nombre_fichero"] = archivo.split(".")[0]
  else:
    df_tmp = pd.read_csv(raiz+archivo+sufijo, index_col=0)
    df_tmp["fichero"] = archivo.split(".")[1]
    df_tmp["nombre_fichero"] = archivo.split(".")[0]
    df = pd.concat([df,df_tmp], axis=0).reset_index(drop=True)

lim_i = 25
lim_f = 50
df_dev_seleccion = pd.read_csv(raiz+"dataset_dev.csv", index_col=0)
df["fichero"] = df["fichero"].astype(int)
df_dev = pd.merge(df, df_dev_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
print(len(df_dev))
df_dev = df_dev.loc[(df_dev["longitud"] > lim_i) & (df_dev["longitud"] <= lim_f), :].copy()
df_dev["Sentence #"] = df_dev["Sentence #"].astype(str) + "_" + df_dev["fichero"].astype(str)  + "_" + df_dev["nombre_fichero"]
print(len(df_dev))
df_ingles_dev = df_dev.loc[df_dev["nombre_fichero"] == "cord"]
df_espanol_dev = df_dev.loc[df_dev["nombre_fichero"] != "cord"]


6341
1529


In [46]:
train_data[3]

(tensor([1223,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0], dtype=torch.uint8),
 tensor([2, 2, 6, 1, 2, 6, 4, 2, 2, 6, 1, 6, 2, 5, 6, 2, 1, 6, 1, 6, 2, 2, 6, 6,
         1, 1, 4, 2, 2, 6, 2, 1, 4, 2, 2, 2, 6, 2, 2, 6, 6, 4, 2, 2, 6, 6, 6, 1,
         2, 2]))

In [47]:
# Seleccionar las oraciones que forman cada dataframe. '_random.csv'
sufijos = [ '_random.csv','_sum_freq_palabra.csv','_sum_freq_pos.csv', '_dif_wv_33.csv']
BS = 32
NUM_EPOCHS = 600 
num_reg = [1200, 900, 600, 300]
for suf in sufijos:
  for num in num_reg:
    resultado = "resultado_"+str(num)+suf
    df_seleccion = pd.read_csv(raiz+str(num)+suf, index_col=0)
    df["fichero"] = df["fichero"].astype(int)
    df_train = pd.merge(df, df_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
    print(len(df_train))
    df_train = df_train.loc[(df_train["longitud"] > lim_i) & (df_train["longitud"] <= lim_f), :].copy()
    df_train["Sentence #"] = df_train["Sentence #"].astype(str) + "_" + df_train["fichero"].astype(str) + "_" + df_train["nombre_fichero"]
    print(len(df_train))
    df_ingles_analiza = df_train.loc[df_train["nombre_fichero"] == "cord"]
    df_espanol_analiza = df_train.loc[df_train["nombre_fichero"] != "cord"]

    # Entrenar con los distintos dataframes
    modelo_en = str(num) + "_bert_en" + str(lim_f) + suf.split(".")[0]
    modelo_es = str(num) + "_bert_es" + str(lim_f)  + suf.split(".")[0]
    raiz = "/content/drive/MyDrive/COLAB - TFM/"
    log_file = "/content/drive/MyDrive/COLAB - TFM/full_training"+ suf 
    df_cruce_ingles = df_ingles_analiza
    df_cruce_espanol = df_espanol_analiza
    for data, modelo, MAX_LEN, lan, data_dev in ([df_cruce_espanol.loc[:,["Sentence #", "Word", "POS", "Tag"]], raiz + modelo_es, lim_f, "es", df_espanol_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]],                                
                                  [df_cruce_ingles.loc[:,["Sentence #", "Word", "POS", "Tag"]],raiz+modelo_en, lim_f, "en", df_ingles_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]]):
      with open(log_file, "a+") as f:
        f.write(";".join([str(datetime.datetime.now()), suf, modelo, "\n"]))
      if len(data) > 0:
        print(modelo)
        print(len(data))
        tag2idx = {'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7}
        tags_vals = ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
        train_data, train_sampler, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, word2idx = bert_formating_one_dataset(data, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS)
        print(tag2idx, tags_vals)
        valid_data, valid_sampler, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, word2idx = bert_formating_one_dataset(data_dev, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS,word2idx=word2idx)
        print(tag2idx, tags_vals)
        if os.path.exists(modelo):
          with open(log_file, "a+") as f:
            f.write("Escojo modelo existente: " +  modelo +"\n")
          model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= False, save_path = modelo)
        else:
          model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True)
        training(model, optimizer,train_dataloader, valid_dataloader, tags_vals, idx2word, epochs = NUM_EPOCHS,
                    max_grad_norm = 1.0, save_path = modelo, early_stopping=NUM_EPOCHS)
        pred_tags, valid_tags, valid_inputs = evaluate(model, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = modelo,
                    guarda_resultado=raiz+resultado+"_validacion_"+str(MAX_LEN)+"_"+lan+".csv")
        pred_tags, valid_tags, valid_inputs = evaluate(model, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, save_path = modelo,
                    guarda_resultado=raiz+resultado+"_entrenamiento_"+str(MAX_LEN)+"_"+lan+".csv")
        os.system("zip "+modelo)
      else:
        print("No hay datos")
    clear_output()


In [ ]:

########################
# INTENTAMOS HACER OVERFITTING DEL SISTEMA
########################
# Seleccionar las oraciones que forman cada dataframe. '_random.csv'
sufijos = ['_sum_freq_palabra.csv','_sum_freq_pos.csv', '_dif_wv_33.csv', '_random.csv']
for suf in sufijos:
  NUM_EPOCHS = 300 
  num_reg = [100,  300, 700, 900, 1200, 1500]
  for num in num_reg:
    resultado = "resultado_"+str(num)+suf
    df_seleccion = pd.read_csv(raiz+str(num)+suf, index_col=0)
    df["fichero"] = df["fichero"].astype(int)
    df_train = pd.merge(df, df_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
    print(len(df_train))
    df_train = df_train.loc[(df_train["longitud"] > lim_i) & (df_train["longitud"] <= lim_f), :].copy()
    df_train["Sentence #"] = df_train["Sentence #"].astype(str) + "_" + df_train["fichero"].astype(str) + "_" + df_train["nombre_fichero"]
    print(len(df_train))
    df_ingles_analiza = df_train.loc[df_train["nombre_fichero"] == "cord"]
    df_espanol_analiza = df_train.loc[df_train["nombre_fichero"] != "cord"]

    # Entrenar con los distintos dataframes
    modelo_en = str(num) + "_bert_en" + str(lim_f) + suf.split(".")[0]
    modelo_es = str(num) + "_bert_es" + str(lim_f)  + suf.split(".")[0]
    raiz = "/content/drive/MyDrive/COLAB - TFM/"
    log_file = "/content/drive/MyDrive/COLAB - TFM/full_training"+ suf 
    df_cruce_ingles = df_ingles_analiza
    df_cruce_espanol = df_espanol_analiza
    for data, modelo, MAX_LEN, lan, data_dev in ([df_cruce_espanol.loc[:,["Sentence #", "Word", "POS", "Tag"]], raiz + modelo_es, lim_f, "es", df_espanol_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]],                                
                                  [df_cruce_ingles.loc[:,["Sentence #", "Word", "POS", "Tag"]],raiz+modelo_en, lim_f, "en", df_ingles_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]]):
      with open(log_file, "a+") as f:
        f.write(";".join([str(datetime.datetime.now()), suf, modelo, "\n"]))
      if len(data) > 0:
        print(modelo)
        print(len(data))
        tag2idx = {'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7}
        tags_vals = ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
        train_data, train_sampler, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs = bert_formating_one_dataset(data, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals)
        print(tag2idx, tags_vals)

        valid_data, valid_sampler, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs = bert_formating_one_dataset(data, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals)
        print(tag2idx, tags_vals)
        if os.path.exists(modelo):
          with open(log_file, "a+") as f:
            f.write("Escojo modelo existente: " +  modelo +"\n")
          model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= False, save_path = modelo)
        else:
          model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True)
        training(model, optimizer, train_dataloader, valid_dataloader, tags_vals, idx2word, epochs = NUM_EPOCHS,
                    max_grad_norm = 1.0, save_path = modelo, early_stopping=300)
        pred_tags, valid_tags, valid_inputs = evaluate(model, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = modelo,
                    guarda_resultado=raiz+resultado+"_validacion_"+str(MAX_LEN)+"_"+lan+".csv")
        pred_tags, valid_tags, valid_inputs = evaluate(model, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, save_path = modelo,
                    guarda_resultado=raiz+resultado+"_entrenamiento_"+str(MAX_LEN)+"_"+lan+".csv")
        os.system("zip "+modelo)
      else:
        print("No hay datos")



3007
1884
/content/drive/MyDrive/COLAB - TFM/100_bert_es50_sum_freq_palabra
1192
{'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7} ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
{'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7} ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Train loss: 2.0403029322624207
Validation loss: 1.5024897456169128
Validation Accuracy: 0.5337019230769231


Epoch:   0%|          | 1/300 [00:01<09:45,  1.96s/it]

F1-Score: 0.007547169811320754


...

Train loss: 0.014073640573769808
Validation loss: 0.0056002705823630095
Validation Accuracy: 0.8313461538461538
F1-Score: 0.8282157676348548


Epoch:  98%|█████████▊| 293/300 [02:44<00:02,  2.80it/s]

Train loss: 0.008664969122037292
Validation loss: 0.005338608549209312
Validation Accuracy: 0.8372115384615384
F1-Score: 0.8278145695364238


Epoch:  98%|█████████▊| 293/300 [02:45<00:03,  1.77it/s]


KeyboardInterrupt: ignored